In [91]:
import pandas as pd
import bs4, requests, time, string
import re
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions



# First we want to replicate Get_and_store_all_players_names_and_ids

recovering the csvs is a bit more unreliable. we use selenium to collect their names, then we do the rest. 

requirements: selenium. Geckodriver. pandas, lxml, bs4, requests. 

In [156]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
import pandas as pd
import string
from time import sleep

class nfl_player_link_scraper:
    def __init__(self):
        self.drv = webdriver.Firefox()
        self.upper_alph = string.ascii_uppercase
        self.paginate_class = 'nfl-o-table-pagination__next'
        self.base_player_df = None
        self.ignored_exceptions = (NoSuchElementException,StaleElementReferenceException,)
        
    # gets player info and links to profiles, from all the players who's last name starts with letter. 
    # returns a dataframe of PlayerName | current Team | Position | Status | Link | 
    def get_curr_players_with_last_name(self,letter):
        #aggregate list is for the pandas concat coming later.
        aggregate_list = []
        url = f"http://www.nfl.com/players/active/{letter}"

        self.drv.get(url)


        # we handle pagination by knowing that they give us the max num of players. Each page renders up to 100 players, so we keep incrementing until we hit max.
        max_players_to_scrape = int(self.drv.find_element_by_xpath('/html/body/div[3]/main/section[2]/div/div/div/header').text.split(' ')[0])
        print(f'preparing to scrape {max_players_to_scrape} players starting with {letter}. \n Selenium will visit {max_players_to_scrape//100+1} pages.\n')

        # We need to screen for players that actually have links. Otherwise, their data isnt useful. 
        WebDriverWait(self.drv, 3,ignored_exceptions=self.ignored_exceptions).until(expected_conditions.presence_of_element_located((By.XPATH, '/html/body/div[3]/main/section[2]/div/div/div/div/table/tbody/tr/td/div/a')))
        link_web_obj = self.drv.find_elements_by_xpath('/html/body/div[3]/main/section[2]/div/div/div/div/table/tbody/tr/td/div/a')

        screening_list = [link.text for link in link_web_obj if link.get_attribute('href') is not None]
        #create dataframe to represent the current page. 
        curr_df = pd.read_html(url)[0]
        curr_df = curr_df[curr_df['Player'].isin(screening_list)]
        curr_df['link'] = [link.get_attribute('href') for link in link_web_obj]
        aggregate_list.append(curr_df)
        
        current_players_scraped = curr_df.shape[0]
        #handle pagination. 
        while current_players_scraped < max_players_to_scrape:
            #click next button. 
            paginate_btn = WebDriverWait(self.drv, 3,ignored_exceptions=self.ignored_exceptions).until(expected_conditions.presence_of_element_located((By.CLASS_NAME, self.paginate_class)))
            paginate_btn.click()
            url = self.drv.current_url

            WebDriverWait(self.drv, 3,ignored_exceptions=self.ignored_exceptions).until(expected_conditions.presence_of_element_located((By.XPATH, '/html/body/div[3]/main/section[2]/div/div/div/div/table/tbody/tr/td/div/a')))
            link_web_obj = self.drv.find_elements_by_xpath('/html/body/div[3]/main/section[2]/div/div/div/div/table/tbody/tr/td/div/a')

            curr_df = pd.read_html(url)[0]
            
            sleep(2)

            screening_list = [link.text for link in link_web_obj if link.get_attribute('href') is not None]
            curr_df = curr_df[curr_df['Player'].isin(screening_list)]
            curr_df['link'] = [link.get_attribute('href') for link in link_web_obj]
            aggregate_list.append(curr_df)

            print(f"Scraped around:{round(current_players_scraped/max_players_to_scrape,2)*100}% of players starting with {letter}")

            if current_players_scraped + curr_df.shape[0] >= max_players_to_scrape:
                return pd.concat(aggregate_list)
            else:
                current_players_scraped+= curr_df.shape[0]
    def get_historical_players_with_last_name(self,letter):
        #aggregate list is for the pandas concat coming later.
        aggregate_list = []
        url = f"http://www.nfl.com/players/retired/{letter}"

        self.drv.get(url)


        # we handle pagination by knowing that they give us the max num of players. Each page renders up to 100 players, so we keep incrementing until we hit max.
        max_players_to_scrape = int(self.drv.find_element_by_xpath('/html/body/div[3]/main/section[2]/div/div/div/header').text.split(' ')[0])
        print(f'preparing to scrape {max_players_to_scrape} players starting with {letter}. \n Selenium will visit {max_players_to_scrape//100+1} pages.\n')


        # We need to screen for players that actually have links. Otherwise, their data isnt useful. 
        awaited = WebDriverWait(self.drv, 3,ignored_exceptions=self.ignored_exceptions).until(expected_conditions.presence_of_element_located((By.XPATH, '/html/body/div[3]/main/section[2]/div/div/div/div/table')))
        link_web_obj = awaited.find_elements_by_tag_name('a')
        screening_list = [link.text for link in link_web_obj if link.get_attribute('href') is not None]
        #create dataframe to represent the current page. 
        curr_df = pd.read_html(url)[0]
        curr_df = curr_df[curr_df['Player'].isin(screening_list)]
        curr_df['link'] = [link.get_attribute('href') for link in link_web_obj]
        aggregate_list.append(curr_df)
        current_players_scraped = curr_df.shape[0]

        #handle pagination. 
        while current_players_scraped < max_players_to_scrape:
            #click next button. 
            paginate_btn = WebDriverWait(self.drv, 3,ignored_exceptions=self.ignored_exceptions).until(expected_conditions.presence_of_element_located((By.CLASS_NAME, self.paginate_class)))
            paginate_btn.click()
            url = self.drv.current_url
            awaited = WebDriverWait(self.drv, 7,ignored_exceptions=self.ignored_exceptions).until(expected_conditions.presence_of_element_located((By.XPATH, '/html/body/div[3]/main/section[2]/div/div/div/div/table/tbody/tr/td/div/a')))
            link_web_obj = awaited.find_elements_by_tag_name('a')
            
            curr_df = pd.read_html(url)[0]

            curr_df = curr_df[curr_df['Player'].isin(screening_list)]
            curr_df['link'] = [link.get_attribute('href') for link in link_web_obj]
            aggregate_list.append(curr_df)
            
            print(f"Scraped around:{round(current_players_scraped/max_players_to_scrape,2)*100}% of players starting with {letter}")

            if current_players_scraped + curr_df.shape[0] >= max_players_to_scrape:
                return pd.concat(aggregate_list)
            else:
                current_players_scraped+= curr_df.shape[0]

    #Void fcn that gets all current_players and writes them to a CSV
    def get_current_player_links_df(self):
        aggregate_list = []
        failure_list = []
        for letter in self.upper_alph:
            try:
                aggregate_list.append(self.get_curr_players_with_last_name(letter))
            except ValueError:
                print(f"value error on {letter}")
                failure_list.append(letter)

        #value errors are really hard to replicate, might be due to dom rendering. So we do a second pass. 
        for failure in failure_list:
            try:
                aggregate_list.append(self.get_curr_players_with_last_name(failure))
            except ValueError:
                print("Bug in letter ", letter)

        self.base_player_df = pd.concat(aggregate_list)

        print(f'successfully scraped {self.base_player_df.shape[0]} current players.')

        self.base_player_df.to_csv("../data/curr_player_names_links.csv")
        
        
    #Void fcn that gets all current_players and writes them to a CSV

    def get_historical_player_links_df(self):
        aggregate_list = []
        failure_list = []
        for letter in self.upper_alph:
            try:
                aggregate_list.append(self.get_historical_players_with_last_name(letter))
            except ValueError:
                print(f"value error on {letter}")
                failure_list.append(letter)

        #value errors are really hard to replicate, might be due to dom rendering. So we do a second pass. 
        for failure in failure_list:
            print(f"Rerunning scraping for players of: {failure}")
            try:
                aggregate_list.append(self.get_historical_players_with_last_name(failure))
            except ValueError:
                print("Bug in letter ", letter)

        self.base_player_df = pd.concat(aggregate_list)

        print(f'successfully scraped {self.base_player_df.shape[0]} historical players.')

        self.base_player_df.to_csv("../data/hist_player_names_links.csv")



In [8]:
#first, activate selenium.
driver = webdriver.Firefox()

In [9]:
page = driver.get('http://www.nfl.com/players/active/a')
driver.find_element_by_xpath('/html/body/div[3]/main/section[2]/div/div/div/footer/div/div/a').click()

In [28]:
c = []
player_table = driver.find_elements_by_xpath('/html/body/div[3]/main/section[2]/div/div/div/div/table/tbody/tr/td/div/a')

In [37]:
#pd.read_html('http://www.nfl.com/players/active/a',header=0)[0]
links = [x.get_attribute('href') for x in player_table]

In [39]:
test = pd.read_html('https://www.nfl.com/players/active/a',header=0)[0]
test['links']=links


In [83]:
spider = nfl_player_link_scraper()
spider.get_curr_players_with_last_name('J')

,Player,Current Team,Position,Status,link
0,Kyle Juszczyk,San Francisco 49ers,FB,ACT,https://www.nfl.com/players/kyle-juszczyk/
1,Luke Juriga,Philadelphia Eagles,C,ACT,https://www.nfl.com/players/luke-juriga/
2,Colton Jumper,New Orleans Saints,LB,UFA,https://www.nfl.com/players/colton-jumper/
3,Jalen Julius,Kansas City Chiefs,SAF,ACT,https://www.nfl.com/players/jalen-julius/
4,Matt Judon,Baltimore Ravens,OLB,ACT,https://www.nfl.com/players/matt-judon/
...,...,...,...,...,...
69,Myles Jack,Jacksonville Jaguars,OLB,ACT,https://www.nfl.com/players/myles-jack/
70,Joe Jackson,Cleveland Browns,DE,ACT,https://www.nfl.com/players/joe-jackson/
71,Mike Jackson,New England Patriots,CB,DEV,https://www.nfl.com/players/mike-jackson/
72,Branden Jackson,Seattle Seahawks,DE,RES,https://www.nfl.com/players/branden-jackson/


In [74]:
oof = [x.text for x in spider.drv.find_elements_by_xpath('/html/body/div[3]/main/section[2]/div/div/div/div/table/tbody/tr/td/div/a') if x.get_attribute('href') is not None]
curr_table = pd.read_html(spider.drv.current_url,header=0)[0]

In [157]:
scraper = nfl_player_link_scraper()

preparing to scrape 1275 players starting with A. 
 Selenium will visit 13 pages.



ValueError: Length of values (0) does not match length of index (100)

In [163]:
f = WebDriverWait(scraper.drv, 3,ignored_exceptions=scraper.ignored_exceptions).until(expected_conditions.presence_of_element_located((By.XPATH, '/html/body/div[3]/main/section[2]/div/div/div/div/table')))
d=f.find_elements_by_tag_name('a')

In [165]:
[x.get_attribute('href') for x in d]


['https://www.nfl.com/players/bill-anderson/',
 'https://www.nfl.com/players/bill-anderson-3/',
 'https://www.nfl.com/players/bill-andrews/',
 'https://www.nfl.com/players/bill-applegate/',
 'https://www.nfl.com/players/bill-armstrong/',
 'https://www.nfl.com/players/bill-armstrong-2/',
 'https://www.nfl.com/players/bill-ashbaugh/',
 'https://www.nfl.com/players/bill-atessis/',
 'https://www.nfl.com/players/bill-athey/',
 'https://www.nfl.com/players/bill-atkins/',
 'https://www.nfl.com/players/bill-austin/',
 'https://www.nfl.com/players/bill-austin-2/',
 'https://www.nfl.com/players/bill-ayre/',
 'https://www.nfl.com/players/billy-alford/',
 'https://www.nfl.com/players/billy-allen/',
 'https://www.nfl.com/players/billy-alsbrooks/',
 'https://www.nfl.com/players/billy-anderson-2/',
 'https://www.nfl.com/players/billy-anderson/',
 'https://www.nfl.com/players/billy-andrews/',
 'https://www.nfl.com/players/billy-ard/',
 'https://www.nfl.com/players/billy-atkins/',
 'https://www.nfl.com